<h1><center>  主题转移学习 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取</center></h1>

 


## 目标: 
-	提取参考数据集的主题并测量验证数据集中的关键指标（强度，情绪，共识，暴露）


## 数据:
-	任何两个数据集，无论是按时间排序还是通过其他方法选择

    **Nucleus Datafeed可直接调用主要中央银行的所有内容**


## Nucleus APIs used:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	主题转移 API
 - 	*api_instance.post_topic_transfer_api(payload)*


-	主题情感转移 API
 - 	*api_instance.post_topic_sentiment_transfer_api(payload)*


-	主题共识转移 API
 - 	*api_instance.post_topic_consensus_transfer_api(payload)*


## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
# 利用自己的语库
print('---- Case 1: you are using your own corpus, coming from a local folder ----')
folder = 'News_feed'         
dataset = 'News_feed'# str | Destination dataset where the file will be inserted.

# 以递归方式从文件夹构建文件。 
# 每一项采用以下格式：
# {'filename': filename,   # 要上传的文件名。 需要
#  'metadata': {           # 该文件的元数据。 可选
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
        file_dict = {'filename': os.path.join(root, file),
                     'metadata': {'source': 'Tech Crunch',
                                  'author': 'Sarah Moore'
                                  'category': 'Media',
                                  'date': '2019-01-01'}}
        file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

    
# 利用Nucleus自带Feed
print('---- Case 2: you are using an embedded datafeed ----')
dataset = 'sumup/rss_feed_finance'# 在Nucleus中调用数据。


-	您可以仅保留在选定的回顾期间发布的文档的子集

**这可以直接在执行内容分析的API中完成，如下所示**



### 2.	转移学习
-	提取参考数据集上的关键主题


-	在验证数据集上测量每个主题的强度，情绪和共识


-   测量验证数据集中每个文档对每个主题的曝光和情感贡献


-	使用用户可用的不同参数优化转移学习


In [ ]:
print('------------------- Get topic transfer -----------------------')

payload = nucleus_api.TopicTransferModel(dataset0='News_feed', 
                                         dataset1="test_feed",
                                        query='', 
                                        custom_stop_words='', 
                                        num_topics=8, 
                                        num_keywords=8,
                                        metadata_selection='')
try:
    api_response = api_instance.post_topic_transfer_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    doc_ids_t1 = api_response.result.doc_ids_t1
    topics = api_response.result.topics
    for i,res in enumerate(topics):
        print('Topic', i, 'exposure within validation dataset:')
        print('    Keywords:', res.keywords)
        print('    Strength:', res.strength)
        print('    Document IDs:', doc_ids_t1)
        print('    Exposure per Doc in Validation Dataset:', res.doc_topic_exposures_t1)
        print('---------------')
    
print('-------------------------------------------------------------')

-	对主题情感转移或/和主题共识转移重复上述任务，具体取决于您希望从参考转移到验证的分析的哪个方面

In [ ]:
print('------------------- Get topic sentiment transfer -----------------------')

payload = nucleus_api.TopicSentimentTransferModel(dataset0='News_feed', 
                                        query='', 
                                        custom_stop_words='', 
                                        num_topics=8, 
                                        num_keywords=8,
                                        period_0_start='2018-08-12',
                                        period_0_end='2018-08-15',
                                        period_1_start='2018-08-16',
                                        period_1_end='2018-08-19',
                                        metadata_selection='')
try:
    api_response = api_instance.post_topic_sentiment_transfer_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    topics = api_response.result
    for i,res in enumerate(topics):
        print('Topic', i, 'exposure within validation dataset:')
        print('    Keywords:', res.keywords)
        print('    Strength:', res.strength)
        print('    Sentiment:', res.sentiment)
        print('    Document IDs:', res.doc_ids_t1)
        print('    Sentiment per Doc in Validation Dataset:', res.doc_sentiments_t1)
        print('---------------')
    
print('-------------------------------------------------------------')

In [ ]:
print('------------------- Get topic consensus transfer -----------------------')

payload = nucleus_api.TopicConsensusTransferModel(dataset0='News_feed', 
                                        query='', 
                                        custom_stop_words='', 
                                        num_topics=8, 
                                        num_keywords=8,
                                        period_0_start='2018-08-12',
                                        period_0_end='2018-08-15',
                                        period_1_start='2018-08-16',
                                        period_1_end='2018-08-19',
                                        metadata_selection='')
try:
    api_response = api_instance.post_topic_consensus_transfer_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    topics = api_response.result
    for i,res in enumerate(topics):
        print('Topic', i, 'exposure within validation dataset:')
        print('    Keywords:', res.keywords)
        print('    Consensus:', res.consensus)
        print('---------------')
    
print('-------------------------------------------------------------')

### 3.	结果解释
-	参考和验证数据集上的度量标准之间可能的比较，或使用验证数据集上的度量标准生成生成信号

### 4.	微调

#### a.	量身定制主题

-   排除不相关或次要主题，以便通过使用Topic Transfer API中的`custom stop_words`参数来定制您的转移学习


-	提取参考文档中的关键主题并打印其关键字




In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='News_feed',                         
                            query='',                       
                            num_topics=8, 
                            num_keywords=8,
                            metadata_selection=metadata_selection,
                            period_start='2018-08-12',
                            period_end='2018-08-15')
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:    
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

然后，您可以通过创建custom_stop_words变量来定制传输学习。 如下所示初始化变量，并将其传递到第2节主代码的有效负载中：

In [ ]:
custom_stop_words = ["conference","interview"] # str | 停用词列表。 （可选）

#### b.	将转移学习集中在某些科目上
如果您决定关注转移学习，例如关注政策和宏观经济主题，只需将第2节主要代码中的查询变量替换为：

In [1]:
query = '(inflation OR growth OR unemployment OR stability OR regulation)' # str | Fulltext query, using mysql MATCH boolean query format. Example: "(word1 OR word2) AND (word3 OR word4)" (optional)

#### c.	验证数据集的替代规范
**validation dataset**: 有两种可能的方法。

1) 如果参考和验证数据集是按时间排序的，
 - 将属于验证数据集的文档附加到参考数据集
 - 使用时间选择器来定义哪个时间段是引用，哪个是验证

2) 如果参考和验证数据集不一定是时间排序的，
 - 将两个不同的数据集传递给Topic Transfer API。
 - dataset0将是您的参考数据集，dataset1将是验证数据集。

请注意，如果从参考数据集中提取的主题不在验证数据集中，则主题传输可能不会导致任何结果。

#### d. 外部指定主题

您可以强制要转移到验证数据集的主题。 要将它们传递给任何Transfer Learning API，请在有效内容中使用`fixed_topics`可选输入参数。

In [ ]:
# 示例1：用英语决定权重
fixed_topics = [{"keywords":["inflation expectations", "forward rates", "board projections"], "weights":[0.7, 0.2, 0.1]}]

# 示例2：英文，您不提供权重。 然后将使用相等的权重
fixed_topics = [{"keywords":["inflation expectations", "forward rates", "board projections"]}]

# 示例3：中文（如果您的数据集是中文）并且您没有提供权重
fixed_topics = [{"keywords":["操作", "流动性", "基点", "元", "点", "央行", "进一步", "投资"]},
                {"keywords":["认为", "价格", "数据", "调查", "全国", "统计", "金融市场", "要求"]}]


payload = nucleus_api.TopicTransferModel(dataset0='News_feed', 
                                        dataset1="test_feed",
                                        fixed_topics=fixed_topics,
                                        query='', 
                                        custom_stop_words='', 
                                        num_topics=8, 
                                        num_keywords=8,
                                        metadata_selection='')
try:
    api_response = api_instance.post_topic_transfer_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    doc_ids_t1 = api_response.result.doc_ids_t1
    topics = api_response.result.topics
    for i,res in enumerate(topics):
        print('Topic', i, 'exposure within validation dataset:')
        print('    Keywords:', res.keywords)
        print('    Strength:', res.strength)
        print('    Document IDs:', doc_ids_t1)
        print('    Exposure per Doc in Validation Dataset:', res.doc_topic_exposures_t1)
        print('---------------')

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 